<img src="Maslow's_Hierarchy_of_Needs.svg.png" height = 600px>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import urllib.parse # combine URL components into URL string
import wikipediaapi # query wikipedia through api

from statsmodels.tsa.seasonal import STL # seasonal decompositions
import statsmodels.tsa.stattools as smt


import pickle #  to serialize and deserialize objects in Python
import requests
from scipy import signal
import warnings
from urllib import request
from bs4 import BeautifulSoup
import json
import pytrends
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import wikipediaapi
import pprint as pp

In [4]:
# 
def create_dataframe(name_file):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file
    return: dataframe with the text file
    """
    df = pd.read_csv(name_file, delimiter="\t", header=None, names=['Topics'])
    df.drop_duplicates(inplace=True)
    return df

# Take only starting from the second word in each row
def remove_space(df):
    return df['Topics'].apply(lambda x: x.strip().replace(' ', '_'))

# Parse the topics into the URL format
def parse_topics_into_df(df, start_time, end_time):
    # change the spaces to underscores
    df['url'] = np.zeros(len(df))
    for index, row in df.iterrows():
        topic_value = row['Topics']
        df.loc[index, 'url'] = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/{topic_value}/monthly/{start_time}/{end_time}'

    return df

# Create a new dataframe with timestamp from starting date to ending date
def create_dataframe_timestamp(starting_date, ending_date):
    df_timestamp = pd.DataFrame()
    df_timestamp['Timestamp'] = pd.date_range(start=starting_date, end=ending_date, freq='MS')
    return df_timestamp

# Define a function to fetch data from the URL and handle errors
def fetch_and_parse_url(url):
    try:
        request.urlopen(url).read()
        return True
    except request.HTTPError as e:
        if e.code == 404:
            return None  # or any other value or action you prefer for 404 errors
        else:
            return None  # or handle other HTTP errors as needed
    except Exception as e:
        
        return None  # or handle other exceptions as needed

def get_pageviews_wiki(url):
    """
    Gets the weekly pageviews for one Wikipedia page in one language in the desired period
    param: url: url of the Wikipedia page
    param: start_date: beginning of the desired period 
    param: end_date: end of the desired period 
    return: dataframe column with the monthly pageviews
    """
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    site_json=json.loads(soup.text)
    df=pd.DataFrame(site_json['items'])
    df=df['views']
    return df

def scrape_pageviews(df):
    pageview = pd.DataFrame()
    pageview['Timestamp'] = create_dataframe_timestamp('2019-01-01', '2020-07-31')['Timestamp']

    # Loop through the rows of the DataFrame and append the results of the function to the DataFrame
    for index, row in df.iterrows():
        url = row['url']
        if fetch_and_parse_url(url):
            pageview[row['Topics']] = get_pageviews_wiki(url)

    return pageview

# Specify the path to your text file
text_file_path = 'Esteem/esteem.txt'

def create_dataframe_pageviews(name_file, start_time = '20190101', end_time = '20200731'):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file, start_time: beginning of the desired period, end_time: end of the desired period
    """
    df_topic = create_dataframe(name_file)
    df_topic['Topics'] = remove_space(df_topic)
    df_topic = parse_topics_into_df(df_topic, start_time, end_time)
    df_pageviews = scrape_pageviews(df_topic)
    df_pageviews.fillna(0, inplace=True)
    return df_pageviews, df_topic

# Read the text file into a DataFrame
df_pageviews, df_topic = create_dataframe_pageviews(text_file_path)
print(type(df_pageviews))
print(df_pageviews.shape)
df_pageviews

/var/folders/yd/s4kdxzb13gz3xbwg60k83tmm0000gn/T/ipykernel_35453/2397408024.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pageview[row['Topics']] = get_pageviews_wiki(url)
/var/folders/yd/s4kdxzb13gz3xbwg60k83tmm0000gn/T/ipykernel_35453/2397408024.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pageview[row['Topics']] = get_pageviews_wiki(url)
/var/folders/yd/s4kdxzb13gz3xbwg60k83tmm0000gn/T/ipykernel_35453/2397408024.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

<class 'pandas.core.frame.DataFrame'>
(19, 292)


/var/folders/yd/s4kdxzb13gz3xbwg60k83tmm0000gn/T/ipykernel_35453/2397408024.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pageview[row['Topics']] = get_pageviews_wiki(url)


,Timestamp,NASA_Exceptional_Engineering_Achievement_Medal,Laureus_Lifetime_Achievement_Award,Filmic_Achievement,TCA_Award_for_Outstanding_Achievement_in_Sports,Latin_Grammy_Lifetime_Achievement_Award,Confidence_Bay,Wechsler_Individual_Achievement_Test,Laurel_Award_for_TV_Writing_Achievement,Computerized_Achievement_Levels_Test,...,Confidence_Man_(band),Lasker-Koshland_Special_Achievement_Award_in_Medical_Science,Confidence_Man_(Lost),"Confidence,_West_Virginia",TrueAchievements,Laurence_Olivier_Award_for_Outstanding_Achievement_in_an_Affiliate_Theatre,Major_achievements_in_roller_hockey_by_nation,Differential_Education_Achievement,Juno_International_Achievement_Award,Iran_Nuclear_Achievements_Protection_Act
0,2019-01-01,336,757,99,100,376,61,2072,264,58,...,4139,131,1018,69,355,284,84,255,61,111
1,2019-02-01,318,2050,130,102,550,52,2264,250,50,...,2533,135,1151,69,377,193,95,317,54,107
2,2019-03-01,234,724,104,108,436,77,2764,206,53,...,3371,128,1289,84,350,447,89,434,77,127
3,2019-04-01,259,778,99,97,365,67,3386,228,45,...,2708,94,1119,101,409,530,102,370,68,115
4,2019-05-01,218,780,90,76,391,63,2824,222,40,...,4209,107,1388,57,302,306,92,332,56,135
5,2019-06-01,271,500,91,95,372,45,2048,129,42,...,3628,130,1219,72,413,432,89,261,49,169
6,2019-07-01,406,520,114,89,411,64,1755,156,61,...,2038,112,1097,60,429,305,143,299,75,234
7,2019-08-01,361,595,91,95,577,64,1778,192,43,...,1637,118,1008,48,400,321,88,251,70,177
8,2019-09-01,278,603,91,86,665,53,2222,190,48,...,1535,132,1114,94,400,521,73,184,56,139
9,2019-10-01,280,747,139,113,550,61,3143,189,53,...,1529,167,1201,109,408,402,108,220,65,217


In [3]:
def get_label_in_language(english_label, target_language):
    # Endpoint URL for the Wikidata Query Service
    url = "https://query.wikidata.org/sparql"
    
    # SPARQL query to get the item with the English label and its label in the target language
    query = f'''
    SELECT ?item ?itemLabel WHERE {{
      ?item rdfs:label "{english_label}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],{target_language}". }}
    }}
    LIMIT 1
    '''
    
    # Headers for the request
    headers = {
        'User-Agent': 'MyBot/0.1 (myemail@example.com)',
        'Accept': 'application/sparql-results+json'
    }
    
    # Make the GET request
    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        results = data['results']['bindings']
        if results:
            # Return the item and its label in the target language
            return results[0]['itemLabel']['value']
        else:
            return '_' # No label found for this language
    else:
        # Handle unsuccessful requests
        response.raise_for_status()

def change_Q(name):
    if name.startswith('Q'):
        return ' '
    else:
        return name

def translate_topics(df_topic, lang):
    # Run through all the topics and get the translation in Italien and store it in a new column in the DataFrame
    df_topic['Topics'] = df_topic['Topics'].str.lower()
    df_topic['Italian'] = df_topic['Topics'].apply(lambda x: get_label_in_language(x, lang))
    df_topic['Italian'].replace('No label found for this language', ' ', inplace=True)
    df_topic['Italian'] = df_topic['Italian'].apply(lambda x: change_Q(x))
    return df_topic

df_topic = translate_topics(df_topic, 'it')
# Count the number of topics that have a translation in Spanish
print(df_topic['Italian'].value_counts())
df_topic

Italian
_                       318
                          3
innovazione               1
improvvisazione           1
gamificazione             1
etnobotanica              1
Co-creazione              1
sintesi                   1
avanguardia               1
storytelling              1
scultura                  1
immaginazione             1
imprenditorialità         1
curiosità                 1
originalità               1
collaborazione            1
plasticità cerebrale      1
Name: count, dtype: int64


,Topics,url,Italian
0,innovation,https://wikimedia.org/api/rest_v1/metrics/page...,innovazione
1,imagination,https://wikimedia.org/api/rest_v1/metrics/page...,immaginazione
2,artistic_expression,https://wikimedia.org/api/rest_v1/metrics/page...,_
3,design_thinking,https://wikimedia.org/api/rest_v1/metrics/page...,_
4,creative_writing,https://wikimedia.org/api/rest_v1/metrics/page...,_
...,...,...,...
336,theatricality_in_storytelling,https://wikimedia.org/api/rest_v1/metrics/page...,_
338,designing_for_inclusive_entrepreneurship,https://wikimedia.org/api/rest_v1/metrics/page...,_
339,the_future_of_creativity_in_biodesign,https://wikimedia.org/api/rest_v1/metrics/page...,_
341,design_for_inclusive_technology,https://wikimedia.org/api/rest_v1/metrics/page...,_
